# Orchestrate the nested chat

In [1]:
## import required libraries
import os
from autogen import AssistantAgent, ConversableAgent, UserProxyAgent
from dotenv import load_dotenv


load_dotenv(override=True) # load environment variables from .env file

True

In [2]:
## llm configuration
llm_config ={
                "model": "gpt-4o",
                "api_key": os.getenv("OPENAI_API_KEY")
            }

## Writer Agent

In [3]:
writer = AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

## Task

In [4]:
task = '''
        Write a concise but engaging blogpost about GPUs.
        Make sure the blogpost is within 100 words.
       '''


In [5]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [6]:
print(reply)

**Unlocking the Power of GPUs: The Heartbeat of Modern Technology**

Graphics Processing Units (GPUs) are revolutionizing the tech world, transforming gaming, AI, and more. Originally designed for rendering images and videos, GPUs now accelerate complex computations, making them indispensable in machine learning, scientific simulations, and cryptocurrency mining. Their parallel processing capabilities enable faster data crunching, leading to innovative breakthroughs and immersive visual experiences. With advancements in GPU technology, expect even more powerful and efficient processing, driving the next wave of technological innovation. Dive into the GPU era and witness the future unfold, pixel by pixel, computation by computation.


## Simple review of the post

In [7]:
## reviewer agent
reviewer = AssistantAgent(
    name="Reviewer",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a reviewer. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [8]:
res = reviewer.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Reviewer (to Writer):


        Write a concise but engaging blogpost about GPUs.
        Make sure the blogpost is within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Reviewer):

**Unlocking the Power of GPUs: The Heartbeat of Modern Technology**

Graphics Processing Units (GPUs) are revolutionizing the tech world, transforming gaming, AI, and more. Originally designed for rendering images and videos, GPUs now accelerate complex computations, making them indispensable in machine learning, scientific simulations, and cryptocurrency mining. Their parallel processing capabilities enable faster data crunching, leading to innovative breakthroughs and immersive visual experiences. With advancements in GPU technology, expect even more powerful and efficient processing, driving the next wave of technological innovation. Dive into the GPU era and witness the future unfold, pixel by pixel, computation by computation.

----------

## Adding multi-level reviews using nested chats

In [9]:
SEO_reviewer = AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="As an SEO expert, your role is to analyze and enhance content for optimal search engine performance. "
        "Focus on providing actionable recommendations that boost rankings and drive organic traffic. "
        "Limit your feedback to 3 key points, ensuring they are specific and directly applicable. "
        "Start each review by introducing yourself as an SEO Reviewer.",
)

In [10]:
grammatical_error_reviewer = AssistantAgent(
    name="Grammatical Error Reviewer",
    llm_config=llm_config,
    system_message="As a grammar specialist, your task is to meticulously examine content "
        "for grammatical errors, punctuation mistakes, and style inconsistencies. "
        "Provide up to 3 key points addressing the most significant grammatical issues. "
        "Ensure your feedback is specific, actionable, and includes examples where appropriate. "
        "Begin each review by introducing yourself as a Grammatical Error Reviewer.",
)

In [11]:
ethics_reviewer = AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="As an ethics specialist, your role is to evaluate content for ethical integrity "
        "and identify any potential moral concerns. "
        "Provide up to 3 specific, actionable recommendations to address ethical issues. "
        "Ensure your feedback is concise and directly applicable. "
        "Start each review by introducing yourself as an Ethics Reviewer.",
)

In [12]:
meta_reviewer = AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrating the nested chat

In [13]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": grammatical_error_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [14]:
reviewer.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [15]:
res = reviewer.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Reviewer (to Writer):


        Write a concise but engaging blogpost about GPUs.
        Make sure the blogpost is within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Reviewer):

**Unlocking the Power of GPUs: The Heartbeat of Modern Technology**

Graphics Processing Units (GPUs) are revolutionizing the tech world, transforming gaming, AI, and more. Originally designed for rendering images and videos, GPUs now accelerate complex computations, making them indispensable in machine learning, scientific simulations, and cryptocurrency mining. Their parallel processing capabilities enable faster data crunching, leading to innovative breakthroughs and immersive visual experiences. With advancements in GPU technology, expect even more powerful and efficient processing, driving the next wave of technological innovation. Dive into the GPU era and witness the future unfold, pixel by pixel, computation by computation.

----------

## Get summary

In [16]:
print(res.summary)

**Unlocking the Power of GPUs: Revolutionizing AI and Gaming**

Graphics Processing Units (GPUs) are revolutionizing the tech world, transforming gaming, accelerating AI advancements, and more. Originally designed for rendering images and videos, GPUs now accelerate complex computations, making them indispensable in machine learning, scientific simulations, and cryptocurrency mining. Their parallel processing capabilities enable faster data crunching, leading to innovative breakthroughs and immersive visual experiences.

Moreover, it's crucial to recognize the ethical and environmental impacts of cryptocurrency mining facilitated by GPUs. Significant energy consumption, carbon footprint, and e-waste from hardware upgrades are pressing concerns requiring transparent discussion. 

With advancements in GPU technology, expect even more powerful and efficient processing, driving the next wave of technological innovation. Dive into the GPU era and witness the future unfold, pixel by pixel, c